In [1]:
import numpy as np
import pandas as pd
# import adata
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
import os
import datetime
import argparse
import torch
from utils import *
from torch.utils.data import DataLoader

## Loading data

For more information check this [link](https://openproblems.bio/neurips_docs/about_tasks/task3_joint_embedding/)

In [2]:
# data
# !ls ~/arc_00077/raw_data/phase2-data/joint_embedding/openproblems_bmmc_multiome_phase2
# ! cd ~/arc_00077/raw_data/phase2-data/joint_embedding/openproblems_bmmc_multiome_phase2
! ls /user/leuven/331/vsc33180/arc_00077/raw_data/phase2-data/joint_embedding/openproblems_bmmc_multiome_phase2

openproblems_bmmc_multiome_phase2.censor_dataset.output_mod1.h5ad
openproblems_bmmc_multiome_phase2.censor_dataset.output_mod2.h5ad
openproblems_bmmc_multiome_phase2.censor_dataset.output_solution.h5ad


In [3]:
os.listdir("/user/leuven/331/vsc33180/arc_00077/raw_data/phase2-data/joint_embedding/openproblems_bmmc_multiome_phase2/")

['openproblems_bmmc_multiome_phase2.censor_dataset.output_mod1.h5ad',
 'openproblems_bmmc_multiome_phase2.censor_dataset.output_mod2.h5ad',
 'openproblems_bmmc_multiome_phase2.censor_dataset.output_solution.h5ad']

In [4]:
data_folder_path = "/user/leuven/331/vsc33180/arc_00077/raw_data/phase2-data/joint_embedding/openproblems_bmmc_multiome_phase2/"
files = os.listdir(data_folder_path)
# print(data_folder_path + files[0])
data1 = sc.read_h5ad(data_folder_path + files[0])
data2 = sc.read(data_folder_path + files[1])
# data3 = sc.read(data_folder_path + files[2])

Only considering the two last: ['.output_mod2', '.h5ad'].
Only considering the two last: ['.output_mod2', '.h5ad'].


In [5]:
print(data1)
print(data2)

AnnData object with n_obs × n_vars = 42492 × 13431
    obs: 'batch', 'size_factors'
    var: 'gene_ids', 'feature_types'
    uns: 'dataset_id', 'organism'
    layers: 'counts'
AnnData object with n_obs × n_vars = 42492 × 116490
    obs: 'batch'
    var: 'feature_types'
    uns: 'dataset_id', 'gene_activity_var_names', 'organism'
    obsm: 'gene_activity'
    layers: 'counts'


In [6]:
# print(data1.var["feature_types"])
rna = pd.DataFrame.sparse.from_spmatrix(data1.X) # 42492 x 13431
rna.columns = data1.var.index #data1.var["gene_ids"] -> ENSEMBL IDs

sc.pp.highly_variable_genes(data1, n_top_genes=5000) # Find HGVs
rna = rna.transpose().loc[data1.var["highly_variable"] == True].transpose()
rna.head() # 42492 x 5000

,LINC01128,KLHL17,ISG15,C1orf159,CDK11B,SLC35E2B,CDK11A,PEX10,PLCH2,TPRG1L,...,MT-ATP8,MT-ATP6,MT-CO3,MT-ND3,MT-ND4L,MT-ND4,MT-ND5,MT-ND6,MT-CYB,AL592183.1
0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,1.688304,0.000000,1.688304,0.000000,0.000000,1.688304,0.0
1,0.0,0.0,0.0,0.0,1.161511,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,1.161511,0.000000,0.000000,0.000000,0.000000,0.000000,2.651025,0.0
2,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.344029,0.599516,1.455109,0.344029,0.344029,0.599516,0.802834,0.344029,1.455109,0.0
3,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,1.355826,2.527316,0.000000,0.000000,0.000000,1.911013,0.000000,0.000000,0.0
4,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,1.556855,2.646429,1.054988,0.000000,1.556855,1.054988,0.000000,2.337923,0.0


In [7]:
atac = pd.DataFrame.sparse.from_spmatrix(data2.X)
atac.columns = data2.var.index
atac.head() # 42492 x 116490

,chr1-9776-10668,chr1-180726-181005,chr1-181117-181803,chr1-191133-192055,chr1-267562-268456,chr1-629497-630394,chr1-633515-634474,chr1-778276-779191,chr1-816868-817761,chr1-822804-823597,...,GL000195.1-137376-138301,GL000219.1-39933-40839,GL000219.1-42172-43054,GL000219.1-44703-45584,GL000219.1-45726-46450,GL000219.1-90062-90937,GL000219.1-99257-100160,KI270726.1-27152-28034,KI270713.1-21434-22336,KI270713.1-29629-30491
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Setting up the necessary arguments and functions

In [8]:
# %tb
# # Model Settings =================================================================================================
# parser = argparse.ArgumentParser(description='Gen-RKM Model')

# parser.add_argument('--N', type=int, default=1000, help='Total # of samples')
# parser.add_argument('--mb_size', type=int, default=300, help='Mini-batch size. See utils.py')
# parser.add_argument('--h_dim', type=int, default=2, help='Dim of latent vector') # shared latent space
# parser.add_argument('--capacity', type=int, default=32, help='Capacity of network. See utils.py') # important in CNNs
# parser.add_argument('--x_fdim', type=int, default=128, help='Input x_fdim. See utils.py') # feature map dimensionality
# parser.add_argument('--y_fdim', type=int, default=20, help='Input y_fdim. See utils.py') # feature map dimensionality
# parser.add_argument('--c_accu', type=float, default=100, help='Input weight on recons_error')

# # Training Settings =============================
# parser.add_argument('--lr', type=float, default=1e-4, help='Input learning rate for optimizer')
# parser.add_argument('--max_epochs', type=int, default=100, help='Input max_epoch for cut-off')
# # parser.add_argument('--device', type=str, default='cuda', help='Device type: cuda or cpu')
# parser.add_argument('--device', type=str, default='cuda' if torch.cuda.is_available() else 'cpu', help='Device type: cuda or cpu')
# parser.add_argument('--workers', type=int, default=0, help='# of workers for dataloader')
# parser.add_argument('--shuffle', type=bool, default=True, help='shuffle dataset: true or false')

# opt = parser.parse_args()
args = {
    "N": rna.shape[0],
#     "mb_size": 300,
    "h_dim": 32,
    "capacity": 32,
    "x_fdim": 128,
    "y_fdim": 256,
    "c_accu": 100,
    "lr": 1e-6,
    "max_epochs": 1,
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    "workers": 15,
    "shuffle": True,
    "batch_size": 100,
}
opt = args
# print(opt["mb_size"])

In [9]:
def kPCA(X, Y):
#     print(X)
#     print(Y)
    a = torch.mm(X, torch.t(X)) + torch.mm(Y, torch.t(Y))
    nh1 = a.size(0)
    oneN = torch.div(torch.ones(nh1, nh1), nh1).float().to(opt["device"])
    a = a - torch.mm(oneN, a) - torch.mm(a, oneN) + torch.mm(torch.mm(oneN, a), oneN)  # centering
    print(a)
#     print(a, end="\r")
    h, s, _ = torch.svd(a, some=False)
    return h[:, :opt["h_dim"]], s

In [10]:
# Energy function
def rkm_loss(output1, X, output2, Y):
    h, s = kPCA(output1, output2)
    U = torch.mm(torch.t(output1), h)
    V = torch.mm(torch.t(output2), h)

    x_tilde = net3(torch.mm(h, torch.t(U))) # NET3
    y_tilde = net4(torch.mm(h, torch.t(V))) # NET4

    # Costs
    f1 = torch.trace(torch.mm(torch.mm(output1, U), torch.t(h))) + torch.trace(
        torch.mm(torch.mm(output2, V), torch.t(h)))
    f2 = 0.5 * torch.trace(torch.mm(h, torch.mm(torch.diag(s[:opt["h_dim"]]), torch.t(h))))
    f3 = 0.5 * ((torch.trace(torch.mm(torch.t(U), U))) + (torch.trace(torch.mm(torch.t(V), V))))
    recon_loss = torch.nn.MSELoss()
    f4 = recon_loss(x_tilde, X) + recon_loss(y_tilde, Y)  # reconstruction loss

    loss = - f1 + f3 + f2 + 0.5 * (- f1 + f3 + f2) ** 2 + opt["c_accu"] * f4  # stabilized loss
    return loss

In [11]:
class create_dirs:
    """ Creates directories for Checkpoints and saving trained models """

    def __init__(self, ct):
        self.ct = ct
        self.dircp = 'checkpoint.pth_{}.tar'.format(self.ct)
        self.dirout = 'Mul_trained_RKM_{}.tar'.format(self.ct)

    def create(self):
        if not os.path.exists('cp/'):
            os.makedirs('cp/')

        if not os.path.exists('out/'):
            os.makedirs('out/')

    def save_checkpoint(self, state, is_best):
        if is_best:
            torch.save(state, 'cp/{}'.format(self.dircp))

In [12]:
# https://discuss.pytorch.org/t/dataloader-access-two-items-at-the-same-time/22664
class ConcatDataset(torch.utils.data.Dataset):
    def __init__(self, dataset1, dataset2):
#         self.dataset1 = dataset1 # datasets should be sorted!
#         self.dataset2 = dataset2

        self.dataset1 = torch.tensor(dataset1.values) # datasets should be sorted!
        self.dataset2 = torch.tensor(dataset2.values)

    def __getitem__(self, index):
#         x1 = self.dataset1.loc[index].to_numpy() #loc, que no iloc (puede que iloc funcione)
#         x2 = self.dataset2.loc[index].to_numpy()
        
        x1 = self.dataset1[index].float()
        x2 = self.dataset2[index].float()

        return x1, x2, index

    def __len__(self):
        return len(self.dataset1) # assuming both datasets have same length
    
# print("Hello")
train_loader = DataLoader(
             ConcatDataset(rna, atac),
             batch_size=args["batch_size"], shuffle=args["shuffle"],
             num_workers=args["workers"]
            )
# print("There")

# initial_time = datetime.datetime.now()
# for i, (datax, datay, index) in enumerate(train_loader):
#     if i==0:
#         initial_time_first_iteration = datetime.datetime.now()
#     print(i)
#     print(datax.shape)
#     print(datay.shape)
#     if i==2:
#         break
        
# print("It took {} min overall.".format(datetime.datetime.now()-initial_time))
# print("It took {} min from the first iteration.".format(datetime.datetime.now()-initial_time_first_iteration))

# num_workers = 5 -> It took 0:08:38.815087 min.
# num_workers = 15 -> It took 0:11:13.346173 min overall - It took 0:01:15.955870 min from the first iteration.
# num_workers = 24 -> It took 0:16:11.308331 min overall - It took 0:02:02.066693 min from the first iteration
# num_workers = 48 -> It took 0:40:50.197775 min overall - It took 0:04:59.826997 min from the first iteration
# Estos 4 de arriba se han hecho con pandas df, los de abajo con tensores
# num_workers = 0 -> It took 0:00:20.585820 min overall - It took 0:00:13.921048 min from the first iteration
# num_workers = 5 -> It took 0:00:35.032580 min overall - It took 0:00:12.420856 min from the first iteration
# num_workers = 15 -> It took 0:00:47.020544 min overall - It took 0:00:05.366418 min from the first iteration
# num_workers = 24 -> It took 0:02:04.082042 min overall - It took 0:00:07.720705 min from the first iteration.
# num_workers = 24 -> It took 0:01:13.212152 min overall - It took 0:00:17.465964 min from the first iteration.
# num_workers = 48 -> It took 0:03:54.132624 min overall - It took 0:00:42.083046 min from the first iteration

## Setting up the NNs

In [13]:
class Net1(nn.Module): # Phi 1 - RNA to feature map
    def __init__(self, input_dim, hidden_dim, args):
        super(Net1, self).__init__()
#         c = capacity
        self.linear1 = nn.Linear(in_features=input_dim,out_features=hidden_dim)
        self.linear2 = nn.Linear(in_features=hidden_dim,out_features=args["x_fdim"])


    def forward(self, x):
        x = self.linear1(x)
        x = F.leaky_relu(x, negative_slope=0.2)
        x = self.linear2(x)
#         x = F.leaky_relu(x, negative_slope=0.2)

        return x   

class Net3(nn.Module):
    def __init__(self, input_dim, hidden_dim, args): #Psi 1 - Feature map to RNA (input dim = 5000)
        super(Net3, self).__init__()
        self.linear2t = nn.Linear(in_features=args["x_fdim"],out_features=hidden_dim)
        self.linear1t = nn.Linear(in_features=hidden_dim,out_features=input_dim)

    def forward(self, x):
        x = self.linear2t(x)
        x = F.leaky_relu(x, negative_slope=0.2)
        x = self.linear1t(x)
        x = F.relu(x) # RNA preprocessed has a lower bound of 0

        return x    

In [14]:
class Net2(nn.Module): # Phi 2 - ATAC to feature map
    def __init__(self, input_dim, hidden_dim_list, args): # Expects a 2D list
        super(Net2, self).__init__()
        assert len(hidden_dim_list) == 2
#         c = capacity
        self.linear1 = nn.Linear(in_features=input_dim,out_features=hidden_dim_list[0])
        self.linear2 = nn.Linear(in_features=hidden_dim_list[0], out_features=hidden_dim_list[1])
        self.linear3 = nn.Linear(in_features=hidden_dim_list[1], out_features=args["y_fdim"])


    def forward(self, x):
        x = self.linear1(x)
        x = F.leaky_relu(x, negative_slope=0.2)
        x = self.linear2(x)
        x = F.leaky_relu(x, negative_slope=0.2)
        x = self.linear3(x)        
#         x = F.leaky_relu(x, negative_slope=0.2)

        return x   

class Net4(nn.Module):
    def __init__(self, input_dim, hidden_dim_list, args): #Psi 2 - Feature map to ATAC (input dim = 116490)
        super(Net4, self).__init__()
        assert len(hidden_dim_list) == 2
        self.linear3t = nn.Linear(in_features=args["y_fdim"], out_features=hidden_dim_list[1])
        self.linear2t = nn.Linear(in_features=hidden_dim_list[1], out_features=hidden_dim_list[0])
        self.linear1t = nn.Linear(in_features=hidden_dim_list[0], out_features=input_dim)

    def forward(self, x):
        x = self.linear3t(x)
        x = F.leaky_relu(x, negative_slope=0.2)
        x = self.linear2t(x)
        x = F.leaky_relu(x, negative_slope=0.2)
        x = self.linear1t(x)
        x = torch.sigmoid(x) # ATAC is has binary values

        return x    

In [15]:
def final_compute(data, net1, net2, kPCA, device=torch.device("cuda" if torch.cuda.is_available() else "cpu")):
    """ Function to compute embeddings of full dataset. """
    # Data is a dictionary with keys RNA and ATAC
#     args["shuffle"] = False
#     xt, _, _ = get_mnist_dataloader(args=args)  # loading data without shuffle
    xt = data["RNA"]
    yt = data["ATAC"]
    xtr = net1(xt.to(args.device))
    ytr = net2(yt.to(args.device)) # This was targets at first, but we don't have labels

    h, s = kPCA(xtr, ytr)
    return torch.mm(torch.t(xtr), h), torch.mm(torch.t(ytr), h), h, s

## Initialize nets and parameters

In [16]:
net1 = Net1(input_dim=5000, hidden_dim=1000, args=args).float().to(opt["device"]) # input_dim, hidden_dim, args
net2 = Net2(input_dim=116490, hidden_dim_list=[10000, 2000], args=args).float().to(opt["device"]) # input_dim, hidden_dim_list, args
net3 = Net3(input_dim=5000, hidden_dim=1000, args=args).float().to(opt["device"]) # input_dim, hidden_dim, args
net4 = Net4(input_dim=116490, hidden_dim_list=[10000, 2000], args=args).float().to(opt["device"]) # input_dim, hidden_dim_list, args
# net4 = Net4(input_dim=116490, hidden_dim_list=[10000, 2000], args=args).double().to(opt["device"])

params = list(net1.parameters()) + list(net3.parameters()) + list(net2.parameters()) + list(net4.parameters())
optimizer = torch.optim.Adam(params, lr=opt["lr"], weight_decay=0)

## Train

In [17]:
ct = time.strftime("%Y%m%d-%H%M")
dirs = create_dirs(ct=ct)
dirs.create()

In [18]:
l_cost = 6  # Costs from where checkpoints will be saved
t = 1
cost = np.inf  # Initialize cost
start = datetime.datetime.now()
while cost > 0.2 and t <= opt["max_epochs"]:  # run epochs until convergence
    avg_loss = 0
    for i, (datax, datay, _) in enumerate(train_loader):
        if i < math.ceil(opt["N"] / opt["batch_size"]):
#             print(f'On step {i}', end="\r")
            print(f'On step {i}')
            datax, datay = datax.float().to(opt["device"]), datay.float().to(opt["device"])
            output1 = net1(datax.float())
            output2 = net2(datay.float())
            loss = rkm_loss(output1, datax, output2, datay).float()

            optimizer.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(parameters=params, max_norm=1)
            optimizer.step()
            avg_loss += loss.detach().cpu().numpy()
        
        else:
            break
    cost = avg_loss

    # Remember lowest cost and save checkpoint
    is_best = cost < l_cost
    l_cost = min(cost, l_cost)

    dirs.save_checkpoint({
        'epochs': t + 1,
        'net1_state_dict': net1.state_dict(),
        'net3_state_dict': net3.state_dict(),
        'net2_state_dict': net2.state_dict(),
        'net4_state_dict': net4.state_dict(),
        'l_cost': l_cost,
        'optimizer': optimizer.state_dict()}, is_best)
    print(f"Epoch: {t}, Cost: {cost}")
    t += 1
print('Finished Training in: {}. Lowest cost: {}'.format(str(datetime.datetime.now() - start), l_cost))

On step 0
tensor([[ 0.7026, -0.1300,  0.1406,  ..., -0.0771,  0.0545,  0.0237],
        [-0.1300,  0.9599, -0.2249,  ...,  0.0334, -0.1598, -0.1799],
        [ 0.1406, -0.2249,  0.7324,  ...,  0.0163, -0.0846,  0.0187],
        ...,
        [-0.0771,  0.0334,  0.0163,  ...,  0.4323, -0.0913, -0.0031],
        [ 0.0545, -0.1598, -0.0846,  ..., -0.0913,  0.6327,  0.0411],
        [ 0.0237, -0.1799,  0.0187,  ..., -0.0031,  0.0411,  0.5989]],
       grad_fn=<AddBackward0>)
On step 1
tensor([[ 0.4765,  0.0028,  0.0628,  ..., -0.1020,  0.0726,  0.0232],
        [ 0.0028,  0.5028, -0.0814,  ..., -0.0945, -0.0174, -0.0337],
        [ 0.0628, -0.0814,  0.9231,  ...,  0.2493, -0.1218, -0.0925],
        ...,
        [-0.1020, -0.0945,  0.2493,  ...,  1.1034, -0.1691, -0.0218],
        [ 0.0726, -0.0174, -0.1218,  ..., -0.1691,  0.4794,  0.0519],
        [ 0.0232, -0.0337, -0.0925,  ..., -0.0218,  0.0519,  0.5117]],
       grad_fn=<AddBackward0>)
On step 2
tensor([[ 0.4177, -0.0536,  0.0732,  ...

On step 17
tensor([[ 0.8772,  0.0418,  0.3157,  ..., -0.1343, -0.0737, -0.0302],
        [ 0.0418,  0.3717,  0.0834,  ..., -0.0522, -0.0081,  0.0434],
        [ 0.3157,  0.0834,  0.9268,  ..., -0.1447, -0.0546,  0.1042],
        ...,
        [-0.1343, -0.0522, -0.1447,  ...,  0.7024, -0.0200, -0.0708],
        [-0.0737, -0.0081, -0.0546,  ..., -0.0200,  0.3348, -0.0035],
        [-0.0302,  0.0434,  0.1042,  ..., -0.0708, -0.0035,  0.4280]],
       grad_fn=<AddBackward0>)
On step 18
tensor([[ 7.6233e-01, -1.4089e-01, -1.0006e-01,  ..., -3.8441e-02,
          4.1453e-03, -1.0668e-01],
        [-1.4089e-01,  5.4304e-01,  1.0143e-01,  ...,  1.3953e-01,
          2.5681e-02,  4.7926e-02],
        [-1.0006e-01,  1.0143e-01,  5.8601e-01,  ..., -1.5492e-02,
         -9.8415e-02,  7.1591e-04],
        ...,
        [-3.8441e-02,  1.3953e-01, -1.5492e-02,  ...,  1.0039e+00,
          7.4123e-03, -3.8440e-02],
        [ 4.1453e-03,  2.5681e-02, -9.8415e-02,  ...,  7.4123e-03,
          6.4863e-01,

On step 34
tensor([[ 0.4456,  0.0181,  0.1146,  ...,  0.0285, -0.0363, -0.0212],
        [ 0.0181,  0.4639, -0.0850,  ...,  0.0708,  0.0295,  0.0188],
        [ 0.1146, -0.0850,  0.6742,  ..., -0.0551, -0.0957, -0.0218],
        ...,
        [ 0.0285,  0.0708, -0.0551,  ...,  0.4952, -0.0220,  0.0101],
        [-0.0363,  0.0295, -0.0957,  ..., -0.0220,  0.8103,  0.0769],
        [-0.0212,  0.0188, -0.0218,  ...,  0.0101,  0.0769,  0.4380]],
       grad_fn=<AddBackward0>)
On step 35
tensor([[ 0.4309, -0.0158,  0.0556,  ..., -0.0826, -0.0064,  0.0412],
        [-0.0158,  1.0891, -0.0566,  ...,  0.0627,  0.2490, -0.0460],
        [ 0.0556, -0.0566,  0.4781,  ..., -0.0587, -0.0642,  0.0855],
        ...,
        [-0.0826,  0.0627, -0.0587,  ...,  0.6892,  0.1082, -0.0652],
        [-0.0064,  0.2490, -0.0642,  ...,  0.1082,  0.7755, -0.0236],
        [ 0.0412, -0.0460,  0.0855,  ..., -0.0652, -0.0236,  0.5537]],
       grad_fn=<AddBackward0>)
On step 36
tensor([[ 6.7370e-01, -2.3507e-02, -3

On step 50
tensor([[ 1.7407e+00, -1.3939e-01, -2.5295e-01,  ..., -5.9831e-02,
         -7.2100e-02,  6.3486e-02],
        [-1.3939e-01,  5.3684e-01,  2.7123e-02,  ..., -1.2056e-01,
          1.9468e-02, -1.1296e-02],
        [-2.5295e-01,  2.7123e-02,  8.6174e-01,  ...,  1.8893e-01,
         -1.2137e-03, -4.0227e-02],
        ...,
        [-5.9831e-02, -1.2056e-01,  1.8893e-01,  ...,  8.3102e-01,
          2.1821e-02, -9.2477e-02],
        [-7.2100e-02,  1.9468e-02, -1.2137e-03,  ...,  2.1821e-02,
          5.9720e-01, -3.0807e-04],
        [ 6.3486e-02, -1.1296e-02, -4.0227e-02,  ..., -9.2477e-02,
         -3.0807e-04,  4.6216e-01]], grad_fn=<AddBackward0>)
On step 51
tensor([[ 0.4727,  0.0129, -0.0460,  ..., -0.0891,  0.1146,  0.0072],
        [ 0.0129,  0.5640, -0.0596,  ..., -0.1557, -0.1299,  0.0174],
        [-0.0460, -0.0596,  0.7313,  ..., -0.0108, -0.0585, -0.1261],
        ...,
        [-0.0891, -0.1557, -0.0108,  ...,  1.7244, -0.1224, -0.1507],
        [ 0.1146, -0.1299, -0

On step 66
tensor([[ 0.5641, -0.0502, -0.0609,  ...,  0.0727,  0.0393, -0.1744],
        [-0.0502,  0.7119, -0.0962,  ..., -0.0334, -0.0939, -0.0801],
        [-0.0609, -0.0962,  0.5526,  ..., -0.0347,  0.0941,  0.0310],
        ...,
        [ 0.0727, -0.0334, -0.0347,  ...,  0.5141,  0.0270, -0.2873],
        [ 0.0393, -0.0939,  0.0941,  ...,  0.0270,  0.3621, -0.0142],
        [-0.1744, -0.0801,  0.0310,  ..., -0.2873, -0.0142,  1.9740]],
       grad_fn=<AddBackward0>)
On step 67
tensor([[ 0.9858,  0.0664, -0.1088,  ..., -0.0976, -0.0315, -0.0369],
        [ 0.0664,  0.6355,  0.0342,  ..., -0.0500,  0.0374,  0.0311],
        [-0.1088,  0.0342,  0.4894,  ...,  0.0602, -0.0060, -0.0396],
        ...,
        [-0.0976, -0.0500,  0.0602,  ...,  0.5658,  0.0183, -0.0146],
        [-0.0315,  0.0374, -0.0060,  ...,  0.0183,  0.5710,  0.0736],
        [-0.0369,  0.0311, -0.0396,  ..., -0.0146,  0.0736,  0.7284]],
       grad_fn=<AddBackward0>)
On step 68
tensor([[ 0.5490, -0.0191,  0.1209,  

...

On step 367
tensor([[237.8565, 175.8421,  61.7258,  ...,  28.5079, -72.5926, 124.2847],
        [175.8420, 133.0095,  46.4468,  ...,  21.3781, -54.1718,  93.4956],
        [ 61.7258,  46.4468,  17.4014,  ...,   7.3227, -19.2320,  32.8774],
        ...,
        [ 28.5078,  21.3780,   7.3226,  ...,   4.3320,  -8.8419,  15.0891],
        [-72.5927, -54.1719, -19.2321,  ...,  -8.8419,  22.8831, -38.3365],
        [124.2847,  93.4955,  32.8773,  ...,  15.0891, -38.3365,  66.6213]],
       grad_fn=<AddBackward0>)
On step 368
tensor([[ 221.4462,  -57.3655,  276.8252,  ..., -188.2158,  627.7731,
          103.6779],
        [ -57.3655,   15.6027,  -71.9199,  ...,   49.0331, -162.9894,
          -26.9166],
        [ 276.8252,  -71.9199,  347.6847,  ..., -236.0748,  787.2134,
          130.0533],
        ...,
        [-188.2158,   49.0331, -236.0748,  ...,  161.2134, -535.1042,
          -88.3594],
        [ 627.7731, -162.9894,  787.2134,  ..., -535.1042, 1786.1620,
          294.9103],
       

On step 381
tensor([[   6.1235,   45.4079,   -8.9353,  ...,  -60.2829,    7.1895,
           15.1924],
        [  45.4080,  372.6906,  -73.6180,  ..., -490.5513,   58.3001,
          123.4406],
        [  -8.9352,  -73.6180,   15.0720,  ...,   97.2006,  -11.5588,
          -24.4003],
        ...,
        [ -60.2828, -490.5513,   97.2006,  ...,  649.8080,  -77.3636,
         -163.2144],
        [   7.1895,   58.3001,  -11.5588,  ...,  -77.3636,    9.9308,
           19.4678],
        [  15.1924,  123.4406,  -24.4003,  ..., -163.2144,   19.4678,
           41.6604]], grad_fn=<AddBackward0>)
On step 382
tensor([[ 1.1352e+02, -1.2239e+00,  2.1084e+01,  ...,  1.8718e+02,
          3.4835e+02, -2.1326e+02],
        [-1.2239e+00,  7.0184e-01, -2.5977e-01,  ..., -1.9673e+00,
         -3.7292e+00,  2.3000e+00],
        [ 2.1084e+01, -2.5977e-01,  4.5547e+00,  ...,  3.4994e+01,
          6.5230e+01, -3.9938e+01],
        ...,
        [ 1.8718e+02, -1.9674e+00,  3.4994e+01,  ...,  3.1052e+02,
   

On step 395
tensor([[  20.2567,  -27.1909,   14.9231,  ...,  -33.1550, -106.3085,
            1.2934],
        [ -27.1908,   38.3082,  -20.5006,  ...,   45.8922,  146.8120,
           -1.9022],
        [  14.9231,  -20.5007,   12.0718,  ...,  -24.8893,  -79.9539,
            0.9756],
        ...,
        [ -33.1549,   45.8922,  -24.8892,  ...,   56.6205,  179.1675,
           -2.3090],
        [-106.3085,  146.8120,  -79.9538,  ...,  179.1675,  576.8193,
           -7.5875],
        [   1.2935,   -1.9022,    0.9757,  ...,   -2.3090,   -7.5875,
            0.6631]], grad_fn=<AddBackward0>)
On step 396
tensor([[   2.1883,    5.1447,  -28.7891,  ...,   10.1428,   42.5160,
           42.9420],
        [   5.1447,   16.6579,  -88.9129,  ...,   32.0590,  131.0568,
          132.2091],
        [ -28.7891,  -88.9129,  503.2815,  ..., -181.3568, -742.4956,
         -748.0573],
        ...,
        [  10.1428,   32.0590, -181.3568,  ...,   67.8638,  267.0042,
          269.6433],
        [  42.5

On step 409
tensor([[ 2.2039e+03,  1.1793e+02, -3.4766e+02,  ..., -5.7631e+02,
          3.3141e+02, -9.3711e+00],
        [ 1.1793e+02,  7.1177e+00, -1.8669e+01,  ..., -3.0980e+01,
          1.7821e+01, -5.4138e-01],
        [-3.4766e+02, -1.8669e+01,  5.5635e+01,  ...,  9.0950e+01,
         -5.2250e+01,  1.3911e+00],
        ...,
        [-5.7631e+02, -3.0980e+01,  9.0950e+01,  ...,  1.5157e+02,
         -8.6645e+01,  2.4706e+00],
        [ 3.3141e+02,  1.7821e+01, -5.2250e+01,  ..., -8.6645e+01,
          5.0436e+01, -1.3905e+00],
        [-9.3710e+00, -5.4126e-01,  1.3912e+00,  ...,  2.4706e+00,
         -1.3905e+00,  8.7585e-01]], grad_fn=<AddBackward0>)
On step 410
tensor([[   56.9141,    75.4960,   148.7079,  ...,   188.4346,  -365.0325,
           -12.6473],
        [   75.4960,   102.5967,   200.2292,  ...,   253.7051,  -491.5065,
           -17.0751],
        [  148.7079,   200.2292,   394.1325,  ...,   499.3361,  -966.7744,
           -33.7797],
        ...,
        [  188.4

On step 423
tensor([[   4.7731,  -55.5804,   -9.5897,  ...,   44.0632,    5.3849,
            6.7412],
        [ -55.5804,  742.7122,  128.9683,  ..., -589.2615,  -70.0500,
          -90.7485],
        [  -9.5897,  128.9683,   22.8699,  ..., -102.3789,  -12.1357,
          -15.8645],
        ...,
        [  44.0633, -589.2613, -102.3788,  ...,  468.7112,   55.6584,
           71.9496],
        [   5.3849,  -70.0499,  -12.1357,  ...,   55.6584,    7.2802,
            8.4984],
        [   6.7413,  -90.7485,  -15.8644,  ...,   71.9496,    8.4984,
           11.9421]], grad_fn=<AddBackward0>)
On step 424
tensor([[ 299.3698, -304.2799,  440.0529,  ..., -137.2611, -410.1500,
          381.4967],
        [-304.2799,  310.5964, -448.2423,  ...,  139.9172,  417.6207,
         -388.5533],
        [ 440.0529, -448.2423,  648.7552,  ..., -202.1422, -604.0593,
          561.7679],
        ...,
        [-137.2611,  139.9172, -202.1422,  ...,   63.9997,  188.3962,
         -175.3828],
        [-410.1

In [19]:
# # net1(datax.float())
# output1 = net1(datax.double()).double()
# output2 = net2(datay.double()).double()
# print(output1)
# print(output2)
# loss = rkm_loss(output1, datax, output2, datay).double()
# print(loss)

In [20]:
# # https://discuss.pytorch.org/t/train-simultaneously-on-two-datasets/649
# # class ConcatDataset(torch.utils.data.Dataset):
# #     def __init__(self, *datasets):
# # #         self.datasets = datasets
# #         self.datasets = [torch.tensor(dataset.values) for dataset in datasets]

# #     def __getitem__(self, i):
# #         return tuple(d[i] for d in self.datasets)

# #     def __len__(self):
# #         return min(len(d) for d in self.datasets)

# # import multiprocessing

# # %time
# # https://discuss.pytorch.org/t/dataloader-access-two-items-at-the-same-time/22664
# class ConcatDataset(torch.utils.data.Dataset):
#     def __init__(self, dataset1, dataset2):
# #         self.dataset1 = dataset1 # datasets should be sorted!
# #         self.dataset2 = dataset2

#         self.dataset1 = torch.tensor(dataset1.values) # datasets should be sorted!
#         self.dataset2 = torch.tensor(dataset2.values)

#     def __getitem__(self, index):
# #         print(index)
# #         print(type(index))
# #         print(type(self.dataset1.loc[index].values))
# #         x1 = self.dataset1.loc[index].to_numpy() #loc, que no iloc (puede que iloc funcione)
# #         x2 = self.dataset2.loc[index].to_numpy()
        
#         x1 = self.dataset1[index]
#         x2 = self.dataset2[index]

#         return x1, x2, index

#     def __len__(self):
#         return len(self.dataset1) # assuming both datasets have same length

# print("Hello")
# train_loader = DataLoader(
#              ConcatDataset(rna, atac),
#              batch_size=args["batch_size"], shuffle=args["shuffle"],
#              num_workers=48 #args["workers"]
#             )
# print("There")

# initial_time = datetime.datetime.now()
# for i, (datax, datay, index) in enumerate(train_loader):
#     if i==0:
#         initial_time_first_iteration = datetime.datetime.now()
#     print(i)
#     print(datax.shape)
#     print(datay.shape)
#     if i==2:
#         break
        
# print("It took {} min overall.".format(datetime.datetime.now()-initial_time))
# print("It took {} min from the first iteration.".format(datetime.datetime.now()-initial_time_first_iteration))

# num_workers = 5 -> It took 0:08:38.815087 min.
# num_workers = 15 -> It took 0:11:13.346173 min overall - It took 0:01:15.955870 min from the first iteration.
# num_workers = 24 -> It took 0:16:11.308331 min overall - It took 0:02:02.066693 min from the first iteration
# num_workers = 48 -> It took 0:40:50.197775 min overall - It took 0:04:59.826997 min from the first iteration
# Estos 4 de arriba se han hecho con pandas df, los de abajo con tensores
# num_workers = 0 -> It took 0:00:20.585820 min overall - It took 0:00:13.921048 min from the first iteration
# num_workers = 5 -> It took 0:00:35.032580 min overall - It took 0:00:12.420856 min from the first iteration
# num_workers = 15 -> It took 0:00:47.020544 min overall - It took 0:00:05.366418 min from the first iteration
# num_workers = 24 -> It took 0:02:04.082042 min overall - It took 0:00:07.720705 min from the first iteration.
# num_workers = 24 -> It took 0:01:13.212152 min overall - It took 0:00:17.465964 min from the first iteration.
# num_workers = 48 -> It took 0:03:54.132624 min overall - It took 0:00:42.083046 min from the first iteration